Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

### Logistic Regression Model with SGD

In [5]:
batch_size = 128
L2_beta = 0.01 # beta for L2 loss regularization

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  L2_loss = L2_beta * (tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases))
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + L2_loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run() # v0.12 and later
  # tf.initialize_all_variables().run() # v0.11 and earlier
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 51.907349
Minibatch accuracy: 5.5%
Validation accuracy: 11.6%
Minibatch loss at step 500: 0.940102
Minibatch accuracy: 83.6%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 0.697103
Minibatch accuracy: 82.8%
Validation accuracy: 80.5%
Minibatch loss at step 1500: 0.737018
Minibatch accuracy: 81.2%
Validation accuracy: 80.8%
Minibatch loss at step 2000: 0.640727
Minibatch accuracy: 85.2%
Validation accuracy: 80.1%
Minibatch loss at step 2500: 0.714441
Minibatch accuracy: 83.6%
Validation accuracy: 80.9%
Minibatch loss at step 3000: 0.672891
Minibatch accuracy: 82.8%
Validation accuracy: 81.0%
Test accuracy: 87.9%


### Neutral Network

In [7]:
batch_size = 128
num_nodes = 1024
L2_beta = 0.01 # beta for L2 loss regularization

graph = tf.Graph()
with graph.as_default():
    
  # Input data. We use a placeholder that will be fed
  tf_dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size))  
  tf_labels = tf.placeholder(tf.float32, shape=(None, num_labels))

  # Matrix 1 Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
  biases_1 = tf.Variable(tf.zeros([num_nodes]))

  # Matrix 2 Variables.
  weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits_1 = tf.matmul(tf_dataset, weights_1) + biases_1
  hidden_1 = tf.nn.relu(logits_1)
  logits_2 = tf.matmul(hidden_1, weights_2) + biases_2
  L2_loss = L2_beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + \
                       tf.nn.l2_loss(biases_1) + tf.nn.l2_loss(biases_2))

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits_2, tf_labels)) + L2_loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Prediction
  label_prediction = tf.nn.softmax(logits_2)

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run() # v0.12 and later
  # tf.initialize_all_variables().run() # v0.11 and earlier
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_dataset : batch_data, tf_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, label_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        label_prediction.eval(feed_dict = {tf_dataset: valid_dataset}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(label_prediction.eval(feed_dict = {tf_dataset: test_dataset}), test_labels))

Initialized
Minibatch loss at step 0: 3475.278076
Minibatch accuracy: 13.3%
Validation accuracy: 25.5%
Minibatch loss at step 500: 21.243954
Minibatch accuracy: 88.3%
Validation accuracy: 83.5%
Minibatch loss at step 1000: 0.801272
Minibatch accuracy: 85.2%
Validation accuracy: 82.1%
Minibatch loss at step 1500: 0.734001
Minibatch accuracy: 82.0%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 0.662900
Minibatch accuracy: 86.7%
Validation accuracy: 81.7%
Minibatch loss at step 2500: 0.713857
Minibatch accuracy: 85.2%
Validation accuracy: 83.0%
Minibatch loss at step 3000: 0.645531
Minibatch accuracy: 85.9%
Validation accuracy: 83.0%
Test accuracy: 89.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run() # v0.12 and later
  # tf.initialize_all_variables().run() # v0.11 and earlier
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = np.random.choice(np.arange(10)) # Reduce the number of batches to 10.
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_dataset : batch_data, tf_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, label_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        label_prediction.eval(feed_dict = {tf_dataset: valid_dataset}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(label_prediction.eval(feed_dict = {tf_dataset: test_dataset}), test_labels))

Initialized
Minibatch loss at step 0: 3420.047607
Minibatch accuracy: 13.3%
Validation accuracy: 36.1%
Minibatch loss at step 500: 20.965714
Minibatch accuracy: 100.0%
Validation accuracy: 70.3%
Minibatch loss at step 1000: 0.319944
Minibatch accuracy: 100.0%
Validation accuracy: 73.1%
Minibatch loss at step 1500: 0.175672
Minibatch accuracy: 100.0%
Validation accuracy: 73.2%
Minibatch loss at step 2000: 0.171789
Minibatch accuracy: 100.0%
Validation accuracy: 73.2%
Minibatch loss at step 2500: 0.170314
Minibatch accuracy: 100.0%
Validation accuracy: 73.2%
Minibatch loss at step 3000: 0.169577
Minibatch accuracy: 100.0%
Validation accuracy: 73.2%
Test accuracy: 80.0%


In this extreme case, the model classified the few training sets accurately with an accuracy of 100.0%. However, the classification of validation and test sets was bad, suggesting a problem of overfitting.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

### Dropout in Neural Network
Remember to turn off L2 regularization in this case.

In [10]:
batch_size = 128
num_nodes = 1024
L2_beta = 0 # beta for L2 loss regularization
keep_prob = 0.5 # keep_prob of dropout

graph = tf.Graph()
with graph.as_default():
    
  # Input data. We use a placeholder that will be fed
  tf_dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size))  
  tf_labels = tf.placeholder(tf.float32, shape=(None, num_labels))

  # Matrix 1 Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
  biases_1 = tf.Variable(tf.zeros([num_nodes]))

  # Matrix 2 Variables.
  weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits_1 = tf.matmul(tf_dataset, weights_1) + biases_1
  hidden_1 = tf.nn.relu(logits_1)
  hidden_1_dropout = tf.nn.dropout(hidden_1, keep_prob = keep_prob)
  logits_2 = tf.matmul(hidden_1, weights_2) + biases_2
  logits_2_training = tf.matmul(hidden_1_dropout, weights_2) + biases_2
  L2_loss = L2_beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + \
                       tf.nn.l2_loss(biases_1) + tf.nn.l2_loss(biases_2))

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits_2_training, tf_labels)) + L2_loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

  # Prediction
  label_prediction = tf.nn.softmax(logits_2)

In [11]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run() # v0.12 and later
  # tf.initialize_all_variables().run() # v0.11 and earlier
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_dataset : batch_data, tf_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, label_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        label_prediction.eval(feed_dict = {tf_dataset: valid_dataset}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(label_prediction.eval(feed_dict = {tf_dataset: test_dataset}), test_labels))

Initialized
Minibatch loss at step 0: 508.056885
Minibatch accuracy: 9.4%
Validation accuracy: 35.5%
Minibatch loss at step 500: 25.894566
Minibatch accuracy: 78.1%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 19.429434
Minibatch accuracy: 81.2%
Validation accuracy: 78.6%
Minibatch loss at step 1500: 13.149889
Minibatch accuracy: 75.8%
Validation accuracy: 79.4%
Minibatch loss at step 2000: 11.261938
Minibatch accuracy: 82.0%
Validation accuracy: 79.6%
Minibatch loss at step 2500: 15.167459
Minibatch accuracy: 75.8%
Validation accuracy: 80.2%
Minibatch loss at step 3000: 9.872648
Minibatch accuracy: 84.4%
Validation accuracy: 77.5%
Test accuracy: 84.6%


### Dropout in the extreme overfitting case
Remember to turn off L2 regularization in this case.

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run() # v0.12 and later
  # tf.initialize_all_variables().run() # v0.11 and earlier
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = np.random.choice(np.arange(10)) # Reduce the number of batches to 10.
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_dataset : batch_data, tf_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, label_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        label_prediction.eval(feed_dict = {tf_dataset: valid_dataset}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(label_prediction.eval(feed_dict = {tf_dataset: test_dataset}), test_labels))

Initialized
Minibatch loss at step 0: 368.757172
Minibatch accuracy: 21.1%
Validation accuracy: 27.1%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 71.1%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.9%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.6%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.2%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.8%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 72.5%
Test accuracy: 79.5%


Honestly, dropout did not improve the model and the extreme overfitting case.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


### 4-Layer Model

2 hidden Layers ( Toal 4 layers ) - without convolution - Test Accuracy = 95.8 %

Architecture
3 Layer Neural Network(No convolution) = input-784, hidden1-960, hidden2=650 output=10
L2- Regularization with lambda(regularization parameter) = .0005
Number of steps = 75000
Batch size = 1000

In [13]:
batch_size = 1000
num_nodes_1 = 960
num_nodes_2 = 650
L2_beta = 0.0005 # beta for L2 loss regularization

graph = tf.Graph()
with graph.as_default():
    
  # Input data. We use a placeholder that will be fed
  tf_dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size))  
  tf_labels = tf.placeholder(tf.float32, shape=(None, num_labels))

  # Matrix 1 Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes_1]))
  biases_1 = tf.Variable(tf.zeros([num_nodes_1]))

  # Matrix 2 Variables.
  weights_2 = tf.Variable(tf.truncated_normal([num_nodes_1, num_nodes_2]))
  biases_2 = tf.Variable(tf.zeros([num_nodes_2]))

  # Matrix 3 Variables.
  weights_3 = tf.Variable(tf.truncated_normal([num_nodes_2, num_labels]))
  biases_3 = tf.Variable(tf.zeros([num_labels]))



  # Training computation.
  logits_1 = tf.matmul(tf_dataset, weights_1) + biases_1
  hidden_1 = tf.nn.relu(logits_1)
  logits_2 = tf.matmul(hidden_1, weights_2) + biases_2
  hidden_2 = tf.nn.relu(logits_2)
  logits_3 = tf.matmul(hidden_2, weights_3) + biases_3

    
  L2_loss = L2_beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1) + \
                       tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2) + \
                       tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(biases_3))

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits_3, tf_labels)) + L2_loss
  
  # Optimizer.
  # Optimizer 1
  optimizer = tf.train.GradientDescentOptimizer(0.0005).minimize(loss)
  # Optimizer 2
  #global_step = tf.Variable(0)  # count the number of steps taken.
  #learning_rate = tf.train.exponential_decay(0.0005, global_step, decay_steps = 100, decay_rate = 0.95, staircase=True)
  #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # Prediction
  label_prediction = tf.nn.softmax(logits_3)

In [14]:
num_steps = 7501

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run() # v0.12 and later
  # tf.initialize_all_variables().run() # v0.11 and earlier
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_dataset : batch_data, tf_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, label_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        label_prediction.eval(feed_dict = {tf_dataset: valid_dataset}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(label_prediction.eval(feed_dict = {tf_dataset: test_dataset}), test_labels))

Initialized
Minibatch loss at step 0: 6013.107422
Minibatch accuracy: 7.5%
Validation accuracy: 9.2%
Minibatch loss at step 500: 708.932129
Minibatch accuracy: 76.4%
Validation accuracy: 74.7%
Minibatch loss at step 1000: 610.854126
Minibatch accuracy: 79.8%
Validation accuracy: 76.7%
Minibatch loss at step 1500: 562.949890
Minibatch accuracy: 80.1%
Validation accuracy: 77.7%
Minibatch loss at step 2000: 566.094727
Minibatch accuracy: 79.0%
Validation accuracy: 78.2%
Minibatch loss at step 2500: 504.578430
Minibatch accuracy: 80.5%
Validation accuracy: 78.7%
Minibatch loss at step 3000: 456.334625
Minibatch accuracy: 80.9%
Validation accuracy: 79.1%
Minibatch loss at step 3500: 483.216431
Minibatch accuracy: 79.0%
Validation accuracy: 79.5%
Minibatch loss at step 4000: 451.775391
Minibatch accuracy: 82.2%
Validation accuracy: 79.7%
Minibatch loss at step 4500: 435.783722
Minibatch accuracy: 81.9%
Validation accuracy: 80.0%
Minibatch loss at step 5000: 438.432892
Minibatch accuracy: 80.

### 5-Layer Model

In [55]:
batch_size = 128
num_nodes_1 = 1024
num_nodes_2 = 300
num_nodes_3 = 50
L2_beta = 0.01 # beta for L2 loss regularization

graph = tf.Graph()
with graph.as_default():
    
  # Input data. We use a placeholder that will be fed
  tf_dataset = tf.placeholder(tf.float32, shape=(None, image_size * image_size))  
  tf_labels = tf.placeholder(tf.float32, shape=(None, num_labels))

  # Matrix 1 Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes_1]))
  biases_1 = tf.Variable(tf.zeros([num_nodes_1]))

  # Matrix 2 Variables.
  weights_2 = tf.Variable(tf.truncated_normal([num_nodes_1, num_nodes_2]))
  biases_2 = tf.Variable(tf.zeros([num_nodes_2]))

  # Matrix 3 Variables.
  weights_3 = tf.Variable(tf.truncated_normal([num_nodes_2, num_nodes_3]))
  biases_3 = tf.Variable(tf.zeros([num_nodes_3]))

  # Matrix 4 Variables.
  weights_4 = tf.Variable(tf.truncated_normal([num_nodes_3, num_labels]))
  biases_4 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits_1 = tf.matmul(tf_dataset, weights_1) + biases_1
  hidden_1 = tf.nn.relu(logits_1)
  logits_2 = tf.matmul(hidden_1, weights_2) + biases_2
  hidden_2 = tf.nn.relu(logits_2)
  logits_3 = tf.matmul(hidden_2, weights_3) + biases_3
  hidden_3 = tf.nn.relu(logits_3)
  logits_4 = tf.matmul(hidden_3, weights_4) + biases_4
    
    
    
  L2_loss = L2_beta * (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(biases_1) + \
                       tf.nn.l2_loss(weights_2) + tf.nn.l2_loss(biases_2) + \
                       tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(biases_3) + \
                       tf.nn.l2_loss(weights_4) + tf.nn.l2_loss(biases_4))

  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits_4, tf_labels)) + L2_loss
  
  # Optimizer.
  # Optimizer 1
  #optimizer = tf.train.GradientDescentOptimizer(0.00005).minimize(loss)
  # Optimizer 2
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.0005, global_step,1000,0.90,staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

  # Prediction
  label_prediction = tf.nn.softmax(logits_4)

In [ ]:
num_steps = 60010

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run() # v0.12 and later
  # tf.initialize_all_variables().run() # v0.11 and earlier
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_dataset : batch_data, tf_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, label_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        label_prediction.eval(feed_dict = {tf_dataset: valid_dataset}), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(label_prediction.eval(feed_dict = {tf_dataset: test_dataset}), test_labels))